In [4]:
# Copyright (c) 2021-2023, NVIDIA CORPORATION & AFFILIATES
#
# SPDX-License-Identifier: BSD-3-Clause

"""
Example using operations on the Network object with torch tensors. This can be used to
amortize the cost of finding the best contraction path and autotuning the network across
multiple contractions.

The contraction result is also a torch tensor on the same device as the operands. 
"""

from cuquantum import Network
import numpy
from cuquantum import cutensornet as cutn


# The parameters of the tensor network.
expr = 'ehl,gj,edhg,bif,d,c,k,iklj,cf,a->ba'
shapes = [(8, 2, 5), (5, 7), (8, 8, 2, 5), (8, 6, 3), (8,), (6,), (5,), (6, 5, 5, 7), (6, 3), (3,)]

device = 'cuda'
# Create torch tensors.
operands = [numpy.random.rand(*shape) for shape in shapes]

# Create the network.
with Network(expr, *operands) as n:

    # Find the contraction path.

    print('yooo',n.handle)

    n.optimizer_config_ptr = cutn.create_contraction_optimizer_config(n.handle)
    enum = cutn.ContractionOptimizerConfigAttribute.SIMPLIFICATION_DISABLE_DR
    value = 1
    name = 'SIMPLIFICATION_DISABLE_DR'

    n._set_opt_config_option(name,enum, value)
    
    path, info = n.contract_path({'samples': 500})

    # Autotune the network.
    n.autotune(iterations=5)

    # Perform the contraction.
    r1 = n.contract()
    print("Contract the network (r1):")
    print(r1)

    # Create new operands. 
    operands = [i*operand for i, operand in enumerate(operands, start=1)]

    # Reset the network operands.
    n.reset_operands(*operands)

    # Perform the contraction with the new operands.
    print("Reset the operands and perform the contraction (r2):")
    r2 = n.contract()
    print(r2)

    from math import factorial

    # The operands can also be updated using in-place operations if they are on the GPU.
    for i, operand in enumerate(operands, start=1):
        operand /= i

    #The operands don't have to be reset for in-place operations. Perform the contraction.
    print("Reset the operands in-place and perform the contraction (r3):")
    r3 = n.contract()
    print(r3)

# The context manages the network resources, so n.free() doesn't have to be called.

yooo 94412163173280
Contract the network (r1):
[[14885.78143232  3223.71545214 10754.9003391 ]
 [13916.15361986  3013.72955546 10054.34924366]
 [13873.31312398  3004.45187198 10023.39720626]
 [11234.5209406   2432.98606246  8116.88345842]
 [13315.1794669   2883.58054904  9620.14851655]
 [19864.59472041  4301.94419029 14352.06727078]
 [16139.70104006  3495.26854686 11660.85079094]
 [14943.42436156  3236.19880094 10796.54705829]]
Reset the operands and perform the contraction (r2):
[[5.40175237e+10 1.16982186e+10 3.90273824e+10]
 [5.04989383e+10 1.09362218e+10 3.64852225e+10]
 [5.03434787e+10 1.09025550e+10 3.63729038e+10]
 [4.07678296e+10 8.82881982e+09 2.94545467e+10]
 [4.83181232e+10 1.04639371e+10 3.49095949e+10]
 [7.20846413e+10 1.56108951e+10 5.20807817e+10]
 [5.85677471e+10 1.26836305e+10 4.23148954e+10]
 [5.42266983e+10 1.17435182e+10 3.91785100e+10]]
Reset the operands in-place and perform the contraction (r3):
[[5.40175237e+10 1.16982186e+10 3.90273824e+10]
 [5.04989383e+10 1.0